In [1]:
import random
from amm import amm
import numpy as np

import matplotlib.pyplot as plt
#plt.style.use('paper.mplstyle')

import seaborn as sns
sns.set_theme(style="ticks")

import statsmodels.api as sm
import matplotlib.ticker as mtick
from params import params



def objective(xs_0):
    """ Fix the seed """
    np.random.seed(params['seed'])
    
    """ Initialise the pools """
    Rx0   = params['Rx0']
    Ry0   = params['Ry0']
    phi   = params['phi']
    
    
    
    pools = amm(Rx=Rx0, Ry=Ry0, phi=phi)

    """ Swap and mint """
    l    = pools.swap_and_mint(xs_0)
    """ Simulate 1000 paths of trading in the pools """
    batch_size = params['batch_size']
    T          = params['T']
    kappa      = params['kappa']
    p          = params['p']
    sigma      = params['sigma']

    end_pools, Rx_t, Ry_t, v_t, event_type_t, event_direction_t =\
            pools.simulate( kappa = kappa, p = p, sigma = sigma, T = T, batch_size = batch_size)

    #print('Reserves in asset X for scenario 0:', end_pools[0].Rx)
    #print('Reserves in asset Y for scenario 0:', end_pools[0].Ry)
    x_T = np.zeros(batch_size)
    for k in range(batch_size):
        x_T[k] = np.sum(end_pools[k].burn_and_swap(l))

    ## CVAR:
    x_0      = np.sum(xs_0)
    log_ret  = np.log(x_T) - np.log(x_0)
    r_t = np.mean(log_ret)/T*100
#     print('Average performance     :', r_t)
#     print('Std. Dev. of performance:', np.std(log_ret)/np.sqrt(T)*100)


    # compute cvar
    alpha = params['alpha'] 
    qtl   = -np.quantile(log_ret, 1-alpha)
    cvar  = np.mean(-log_ret[-log_ret>=qtl])
#     print(cvar)
    return cvar, r_t




def derivatives(xs_0):
     #perturbance
     epsilon = 1e-3
     gradients=[]
     cv_1=0
     r_t1=0
     perturbations=np.zeros(params['N_pools'])
     for i in range(params['N_pools']):
         perturbations[i] = epsilon
         cv_1, r_t1= objective(xs_0+perturbations)
         comb1 = cv_1/r_t1
         cv_2, r_t2 =  objective(xs_0-perturbations)
         comb2 = cv_2/r_t2
         gradients_i = (comb1-comb2)/(2*epsilon)
         gradients.append(gradients_i)
         perturbations[i]=0
    # print(cv_1, r_t1)
     return np.array(gradients)
     
def gradient_descent(xs_0):
    learning_rate = 100
    prev_cvar=0
    prev_perf=1
    tol = 0.005
    noise = 1e-2
    while True:
     #   print(xs_0)
        gradients = derivatives(xs_0)
       
        xs_0 = xs_0-learning_rate*gradients
               
        for j in range(params['N_pools']):
            if xs_0[j] <noise:
                xs_0[j] = noise
        scale = np.sum(xs_0)
        xs_0 = xs_0*(10/scale)
        cvar, perf = objective(xs_0)
        if abs(cvar/perf - prev_cvar/prev_perf)<tol:
            return xs_0
        else:
            prev_cvar = cvar
            prev_perf = perf
    return xs_0


xs_0= np.zeros(params['N_pools'])
for i in range(params['N_pools']):
    xs_0[i]=random.random()
    
scale = np.sum(xs_0)

xs_0 = xs_0 * (10/scale)
best_x = gradient_descent(xs_0)

print(best_x)
print(objective(best_x))

C:\Users\anand\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 51.81it/s]


[0.26812384 3.63359566 1.89762382 0.01826357 3.56593037 0.61646274]


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 52.48it/s]

(0.005098792511356218, 0.2666342954386418)


In [2]:
theta = best_x/10
print(theta)

[0.02681238 0.36335957 0.18976238 0.00182636 0.35659304 0.06164627]
